<a href="https://colab.research.google.com/github/DGkurnia/cnkproject/blob/main/%5BClustering%5D_protein_Derfansyah_Guswiranata_Kurnia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pengenalan Diri
## Nama: Derfansyah Guswiranata Kurnia
## Identitas Dicoding : dgkurnia
## Alamat Email Dicoding : weerakurnia@gmail.com

# **1. Perkenalan Dataset**


Tahap pertama, Anda harus mencari dan menggunakan dataset **tanpa label** dengan ketentuan sebagai berikut:

1. **Sumber Dataset**:  
   Dataset dapat diperoleh dari berbagai sumber, seperti public repositories (*Kaggle*, *UCI ML Repository*, *Open Data*) atau data primer yang Anda kumpulkan sendiri.
   
2. **Ketentuan Dataset**:
   - **Tanpa label**: Dataset tidak boleh memiliki label atau kelas.
   - **Jumlah Baris**: Minimal 1000 baris untuk memastikan dataset cukup besar untuk analisis yang bermakna.
   - **Tipe Data**: Harus mengandung data **kategorikal** dan **numerikal**.
     - *Kategorikal*: Misalnya jenis kelamin, kategori produk.
     - *Numerikal*: Misalnya usia, pendapatan, harga.

3. **Pembatasan**:  
   Dataset yang sudah digunakan dalam latihan clustering (seperti customer segmentation) tidak boleh digunakan.

In [ ]:
#Inisiasi Pengunduhan ke Kaggle
!pip install kaggle

In [ ]:
#Instalasi file json
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"derfansyahgk","key":"24be196f622b75f71753aee422847c5c"}'}

In [ ]:
#Penyetalan Lingkungan
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/"

In [ ]:
#Pengunduhan dataset
!kaggle datasets download -d shahir/protein-data-set

Dataset URL: https://www.kaggle.com/datasets/shahir/protein-data-set
License(s): DbCL-1.0
 55% 15.0M/27.4M [00:00<00:00, 79.0MB/s]
100% 27.4M/27.4M [00:00<00:00, 92.9MB/s]


# **2. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [ ]:
#Perpustakaan dasar
import numpy as np
import pandas as pd
#Grafik dasar
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#persiapan 'titik sementara' di google colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Bagian Scaler
from sklearn.preprocessing import StandardScaler #Untuk PCA
from sklearn.preprocessing import RobustScaler #untuk hierarki
#PCA
from sklearn.decomposition import PCA
#ke optimasi

In [ ]:
#inisiasi 'dill' untuk serialisasi
! pip install dill
import dill

In [ ]:
#optimasi fungsi demi kelanjutan eksekusi
from joblib import Parallel, delayed, dump, load
#Lanjutan serialisasi
import multiprocessing
from multiprocessing import Pool
import os #untuk os
import time

In [ ]:
#Perpustakaan untuk protokol 'clustering'
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster #demi 0.55-0.70
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import squareform
from sklearn.metrics import silhouette_score

# **3. Memuat Dataset**

Pada tahap ini, Anda perlu memuat dataset ke dalam notebook. Jika dataset dalam format CSV, Anda bisa menggunakan pustaka pandas untuk membacanya. Pastikan untuk mengecek beberapa baris awal dataset untuk memahami strukturnya dan memastikan data telah dimuat dengan benar.

Jika dataset berada di Google Drive, pastikan Anda menghubungkan Google Drive ke Colab terlebih dahulu. Setelah dataset berhasil dimuat, langkah berikutnya adalah memeriksa kesesuaian data dan siap untuk dianalisis lebih lanjut.

In [ ]:
#Buka dataset
!unzip protein-data-set.zip

Archive:  protein-data-set.zip
  inflating: pdb_data_no_dups.csv    
  inflating: pdb_data_seq.csv        


In [ ]:
#Baca dataset
dfi = pd.read_csv('/content/pdb_data_no_dups.csv')
dfic = dfi.copy()
dfii = pd.read_csv('/content/pdb_data_seq.csv')
dfiic = dfii.copy()

In [ ]:
#cek hasil
dfiic.head(7)

,structureId,chainId,sequence,residueCount,macromoleculeType
0,100D,A,CCGGCGCCGG,20,DNA/RNA Hybrid
1,100D,B,CCGGCGCCGG,20,DNA/RNA Hybrid
2,101D,A,CGCGAATTCGCG,24,DNA
3,101D,B,CGCGAATTCGCG,24,DNA
4,101M,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,Protein
5,102D,A,CGCAAATTTGCG,24,DNA
6,102D,B,CGCAAATTTGCG,24,DNA


# **4. Exploratory Data Analysis (EDA)**

Pada tahap ini, Anda akan melakukan **Exploratory Data Analysis (EDA)** untuk memahami karakteristik dataset. EDA bertujuan untuk:

1. **Memahami Struktur Data**
   - Tinjau jumlah baris dan kolom dalam dataset.  
   - Tinjau jenis data di setiap kolom (numerikal atau kategorikal).

2. **Menangani Data yang Hilang**  
   - Identifikasi dan analisis data yang hilang (*missing values*). Tentukan langkah-langkah yang diperlukan untuk menangani data yang hilang, seperti pengisian atau penghapusan data tersebut.

3. **Analisis Distribusi dan Korelasi**  
   - Analisis distribusi variabel numerik dengan statistik deskriptif dan visualisasi seperti histogram atau boxplot.  
   - Periksa hubungan antara variabel menggunakan matriks korelasi atau scatter plot.

4. **Visualisasi Data**  
   - Buat visualisasi dasar seperti grafik distribusi dan diagram batang untuk variabel kategorikal.  
   - Gunakan heatmap atau pairplot untuk menganalisis korelasi antar variabel.

Tujuan dari EDA adalah untuk memperoleh wawasan awal yang mendalam mengenai data dan menentukan langkah selanjutnya dalam analisis atau pemodelan.

In [ ]:
#penyatuan awal
dfkomb = dfic.merge(dfiic, how='inner', on ='structureId', sort=True).copy()
dfkomb.head(7)

,structureId,classification,experimentalTechnique,macromoleculeType_x,residueCount_x,resolution,structureMolecularWeight,crystallizationMethod,crystallizationTempK,densityMatthews,densityPercentSol,pdbxDetails,phValue,publicationYear,chainId,sequence,residueCount_y,macromoleculeType_y
0,100D,DNA-RNA HYBRID,X-RAY DIFFRACTION,DNA/RNA Hybrid,20,1.90,6360.30,"VAPOR DIFFUSION, HANGING DROP",NaN,1.78,30.89,"pH 7.00, VAPOR DIFFUSION, HANGING DROP",7.0,1994.0,A,CCGGCGCCGG,20,DNA/RNA Hybrid
1,100D,DNA-RNA HYBRID,X-RAY DIFFRACTION,DNA/RNA Hybrid,20,1.90,6360.30,"VAPOR DIFFUSION, HANGING DROP",NaN,1.78,30.89,"pH 7.00, VAPOR DIFFUSION, HANGING DROP",7.0,1994.0,B,CCGGCGCCGG,20,DNA/RNA Hybrid
2,101D,DNA,X-RAY DIFFRACTION,DNA,24,2.25,7939.35,NaN,NaN,2.00,38.45,NaN,NaN,1995.0,A,CGCGAATTCGCG,24,DNA
3,101D,DNA,X-RAY DIFFRACTION,DNA,24,2.25,7939.35,NaN,NaN,2.00,38.45,NaN,NaN,1995.0,B,CGCGAATTCGCG,24,DNA
4,101M,OXYGEN TRANSPORT,X-RAY DIFFRACTION,Protein,154,2.07,18112.80,NaN,NaN,3.09,60.20,"3.0 M AMMONIUM SULFATE, 20 MM TRIS, 1MM EDTA, ...",9.0,1999.0,A,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...,154,Protein
5,102D,DNA,X-RAY DIFFRACTION,DNA,24,2.20,7637.17,"VAPOR DIFFUSION, SITTING DROP",277.0,2.28,46.06,"pH 7.00, VAPOR DIFFUSION, SITTING DROP, temper...",7.0,1995.0,A,CGCAAATTTGCG,24,DNA
6,102D,DNA,X-RAY DIFFRACTION,DNA,24,2.20,7637.17,"VAPOR DIFFUSION, SITTING DROP",277.0,2.28,46.06,"pH 7.00, VAPOR DIFFUSION, SITTING DROP, temper...",7.0,1995.0,B,CGCAAATTTGCG,24,DNA


In [ ]:
#pembersihan kolom null dan na di 'dfkomb' (dataframe kombinasi)
dfkomb = dfkomb.dropna().copy()
dfkomb.isna().sum()

,0
structureId,0
classification,0
experimentalTechnique,0
macromoleculeType_x,0
residueCount_x,0
resolution,0
structureMolecularWeight,0
crystallizationMethod,0
crystallizationTempK,0
densityMatthews,0


In [ ]:
#Tahap awal EDA
dfkomb.info()

<class 'pandas.core.frame.DataFrame'>
Index: 215142 entries, 5 to 470659
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   structureId               215142 non-null  object 
 1   classification            215142 non-null  object 
 2   experimentalTechnique     215142 non-null  object 
 3   macromoleculeType_x       215142 non-null  object 
 4   residueCount_x            215142 non-null  int64  
 5   resolution                215142 non-null  float64
 6   structureMolecularWeight  215142 non-null  float64
 7   crystallizationMethod     215142 non-null  object 
 8   crystallizationTempK      215142 non-null  float64
 9   densityMatthews           215142 non-null  float64
 10  densityPercentSol         215142 non-null  float64
 11  pdbxDetails               215142 non-null  object 
 12  phValue                   215142 non-null  float64
 13  publicationYear           215142 non-null  float6

# **5. Data Preprocessing**

Pada tahap ini, data preprocessing adalah langkah penting untuk memastikan kualitas data sebelum digunakan dalam model machine learning. Data mentah sering kali mengandung nilai kosong, duplikasi, atau rentang nilai yang tidak konsisten, yang dapat memengaruhi kinerja model. Oleh karena itu, proses ini bertujuan untuk membersihkan dan mempersiapkan data agar analisis berjalan optimal.

Berikut adalah tahapan-tahapan yang perlu dilakukan, namun **tidak terbatas** pada:
1. Menghapus atau Menangani Data Kosong (Missing Values)
2. Menghapus Data Duplikat
3. Normalisasi atau Standarisasi Fitur
4. Deteksi dan Penanganan Outlier
5. Encoding Data Kategorikal
6. Binning (Pengelompokan Data)

In [ ]:
#inspeksi tipe data
infodf1 = dfic.info()
infodf2 = dfiic.info()
print("Info dfic")
print(infodf1)
print("info dataset di dfiic")
print(infodf2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141401 entries, 0 to 141400
Data columns (total 14 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   structureId               141401 non-null  object 
 1   classification            141399 non-null  object 
 2   experimentalTechnique     141401 non-null  object 
 3   macromoleculeType         137636 non-null  object 
 4   residueCount              141401 non-null  int64  
 5   resolution                128589 non-null  float64
 6   structureMolecularWeight  141401 non-null  float64
 7   crystallizationMethod     96242 non-null   object 
 8   crystallizationTempK      97039 non-null   float64
 9   densityMatthews           124724 non-null  float64
 10  densityPercentSol         124749 non-null  float64
 11  pdbxDetails               118534 non-null  object 
 12  phValue                   105110 non-null  float64
 13  publicationYear           117602 non-null  f

In [ ]:
#cek data kosong
totkosongi = dfic.isnull().sum().sort_values(ascending=False)
print("Total data kosong di df1")
print(totkosongi)

Total data kosong di df1
crystallizationMethod       45159
crystallizationTempK        44362
phValue                     36291
publicationYear             23799
pdbxDetails                 22867
densityMatthews             16677
densityPercentSol           16652
resolution                  12812
macromoleculeType            3765
classification                  2
structureId                     0
experimentalTechnique           0
residueCount                    0
structureMolecularWeight        0
dtype: int64


In [ ]:
#Pemisah kolom numerik dan non numerik di dfic dan dfiic
dfic_num = dfic.select_dtypes(include=['float64', 'int64']).copy()
dfic_cat = dfic.select_dtypes(include=['object']).copy()
#di dataset dfiic
dfiic_num = dfiic.select_dtypes(include=['float64', 'int64']).copy()
dfiic_cat = dfiic.select_dtypes(include=['object']).copy()

In [ ]:
#cek hasil di dfic dan dfiic
print("kolom numerik yang hilang di dfic")
print(dfic_num.isnull().sum())
print("kolom non numerik yang hilang di dfic")
print(dfic_cat.isnull().sum())
#di dataset dfiic
print("kolom numerik di dfiic yang hilang")
print(dfiic_num.isnull().sum())
print("kolom non numerik di dfiic yang hilang")
print(dfiic_cat.isnull().sum())

kolom numerik yang hilang di dfic
residueCount                    0
resolution                  12812
structureMolecularWeight        0
crystallizationTempK        44362
densityMatthews             16677
densityPercentSol           16652
phValue                     36291
publicationYear             23799
dtype: int64
kolom non numerik yang hilang di dfic
structureId                  0
classification               2
experimentalTechnique        0
macromoleculeType         3765
crystallizationMethod    45159
pdbxDetails              22867
dtype: int64
kolom numerik di dfiic yang hilang
residueCount    0
dtype: int64
kolom non numerik di dfiic yang hilang
structureId              0
chainId                 10
sequence                28
macromoleculeType    34817
dtype: int64


In [ ]:
#Persiapan untuk pengisian data kosong (bagian non numerik di dfic)
dfic_cat = dfic_cat.fillna(dfic_cat.mode().iloc[0])
#di dataset dfiic
dfiic_cat = dfiic_cat.fillna(dfiic_cat.mode().iloc[0])

In [ ]:
#Inspeksi data non numerik
print("kolom non numerik yang hilang di dfic")
print(dfic_cat.isnull().sum())
#di dataset dfiic
print("kolom non numerik yang hilang di dfiic")
print(dfiic_cat.isnull().sum())
#cek sampel hasil di dfic dan dfiic
print("sampel hasil di dfic")
print(dfic_cat.head())
print("sampel hasil di dfiic")
print(dfiic_cat.head())

kolom non numerik yang hilang di dfic
structureId              0
classification           0
experimentalTechnique    0
macromoleculeType        0
crystallizationMethod    0
pdbxDetails              0
dtype: int64
kolom non numerik yang hilang di dfiic
structureId          0
chainId              0
sequence             0
macromoleculeType    0
dtype: int64
sampel hasil di dfic
  structureId         classification experimentalTechnique macromoleculeType  \
0        100D         DNA-RNA HYBRID     X-RAY DIFFRACTION    DNA/RNA Hybrid   
1        101D                    DNA     X-RAY DIFFRACTION               DNA   
2        101M       OXYGEN TRANSPORT     X-RAY DIFFRACTION           Protein   
3        102D                    DNA     X-RAY DIFFRACTION               DNA   
4        102L  HYDROLASE(O-GLYCOSYL)     X-RAY DIFFRACTION           Protein   

           crystallizationMethod  \
0  VAPOR DIFFUSION, HANGING DROP   
1  VAPOR DIFFUSION, HANGING DROP   
2  VAPOR DIFFUSION, HANGING DROP 

In [ ]:
#Ke Bagian numerikal di dua dataset
dfic_num = dfic_num.fillna(dfic_num.median()) #Numerik di dfic
dfiic_num = dfiic_num.fillna(dfiic_num.median()) #Numerik di dfiic

In [ ]:
#cek hasil di kolom numerik (dfic dan dfiic)
print("kolom numerik yang hilang di dfic")
print(dfic_num.isnull().sum())
#di dataset dfiic
print("kolom numerik yang hilang di dfiic")
print(dfiic_num.isnull().sum())
#cek sampel
print("Sampel di dfic_num")
print(dfic_num.head(7)) #dataset dfic
print("Sampel di dfiic_num")
print(dfiic_num.head(7)) #dataset dfiic

kolom numerik yang hilang di dfic
residueCount                0
resolution                  0
structureMolecularWeight    0
crystallizationTempK        0
densityMatthews             0
densityPercentSol           0
phValue                     0
publicationYear             0
dtype: int64
kolom numerik yang hilang di dfiic
residueCount    0
dtype: int64
Sampel di dfic_num
   residueCount  resolution  structureMolecularWeight  crystallizationTempK  \
0            20        1.90                   6360.30                 293.0   
1            24        2.25                   7939.35                 293.0   
2           154        2.07                  18112.80                 293.0   
3            24        2.20                   7637.17                 277.0   
4           165        1.74                  18926.61                 293.0   
5           154        1.84                  18010.64                 293.0   
6            24        2.10                   7502.93                 293.0

# **6. Pembangunan Model Clustering**

## **a. Pembangunan Model Clustering**

Pada tahap ini, Anda membangun model clustering dengan memilih algoritma yang sesuai untuk mengelompokkan data berdasarkan kesamaan. Berikut adalah **rekomendasi** tahapannya.
1. Pilih algoritma clustering yang sesuai.
2. Latih model dengan data menggunakan algoritma tersebut.

In [ ]:
#TPenyatuan kembali dua dataset
dficompl = pd.concat([dfic_cat, dfic_num], axis=1).copy()
dfiicompl = pd.concat([dfiic_cat, dfiic_num], axis=1).copy()

In [ ]:
#Penyatuan dua dataset tersebut
dfgabung = dfic.merge(dfiic, how='inner', on ='structureId', sort=True).copy()
dfgabung = dfgabung.rename({'macromoleculeType_x':'macromoleculeType','residueCount_y':'residueCount'},axis=1) #Ubah nama
#cek kolom
dfgabung.columns

Index(['structureId', 'classification', 'experimentalTechnique',
       'macromoleculeType', 'residueCount_x', 'resolution',
       'structureMolecularWeight', 'crystallizationMethod',
       'crystallizationTempK', 'densityMatthews', 'densityPercentSol',
       'pdbxDetails', 'phValue', 'publicationYear', 'chainId', 'sequence',
       'residueCount', 'macromoleculeType_y'],
      dtype='object')

In [ ]:
#cek informasi kolom
dfgabung.isna().sum()

,0
structureId,0
classification,4
experimentalTechnique,0
macromoleculeType,35770
residueCount_x,0
resolution,21663
structureMolecularWeight,0
crystallizationMethod,156395
crystallizationTempK,153452
densityMatthews,81504


In [ ]:
#Hapus kolom kosong
dfgabungbsh = dfgabung.dropna(subset =['structureId', 'classification', 'experimentalTechnique',
       'macromoleculeType', 'residueCount_x', 'resolution',
       'structureMolecularWeight', 'crystallizationMethod',
       'crystallizationTempK', 'densityMatthews', 'densityPercentSol',
       'pdbxDetails', 'phValue', 'publicationYear', 'chainId', 'sequence',
       'residueCount', 'macromoleculeType_y']).copy()
#lihat kolom hilang pasca pembersihan
dfgabungbsh.isna().sum()

,0
structureId,0
classification,0
experimentalTechnique,0
macromoleculeType,0
residueCount_x,0
resolution,0
structureMolecularWeight,0
crystallizationMethod,0
crystallizationTempK,0
densityMatthews,0


In [ ]:
#inspeksi informasi
dfgabungbsh.info()

<class 'pandas.core.frame.DataFrame'>
Index: 215142 entries, 5 to 470659
Data columns (total 18 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   structureId               215142 non-null  object 
 1   classification            215142 non-null  object 
 2   experimentalTechnique     215142 non-null  object 
 3   macromoleculeType         215142 non-null  object 
 4   residueCount_x            215142 non-null  int64  
 5   resolution                215142 non-null  float64
 6   structureMolecularWeight  215142 non-null  float64
 7   crystallizationMethod     215142 non-null  object 
 8   crystallizationTempK      215142 non-null  float64
 9   densityMatthews           215142 non-null  float64
 10  densityPercentSol         215142 non-null  float64
 11  pdbxDetails               215142 non-null  object 
 12  phValue                   215142 non-null  float64
 13  publicationYear           215142 non-null  float6

In [ ]:
#Pemilihan dataset
dfgabungnum = dfgabungbsh.select_dtypes(include=['float64', 'int64']).copy()
dfgabungnum.info()

<class 'pandas.core.frame.DataFrame'>
Index: 215142 entries, 5 to 470659
Data columns (total 9 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   residueCount_x            215142 non-null  int64  
 1   resolution                215142 non-null  float64
 2   structureMolecularWeight  215142 non-null  float64
 3   crystallizationTempK      215142 non-null  float64
 4   densityMatthews           215142 non-null  float64
 5   densityPercentSol         215142 non-null  float64
 6   phValue                   215142 non-null  float64
 7   publicationYear           215142 non-null  float64
 8   residueCount              215142 non-null  int64  
dtypes: float64(7), int64(2)
memory usage: 16.4 MB


In [ ]:
#Proses scaler
stdscal = StandardScaler() #untuk PCA
rbscal = RobustScaler() #untuk hierarki

In [ ]:
#Persiapan dataset untuk hierarki via Robust
sdfgbngnum = stdscal.fit_transform(dfgabungnum) #untuk PCA
rscaldfgbgnum = rbscal.fit_transform(dfgabungnum) #untuk hierarki

## **b. Evaluasi Model Clustering**

Untuk menentukan jumlah cluster yang optimal dalam model clustering, Anda dapat menggunakan metode Elbow atau Silhouette Score.

Metode ini membantu kita menemukan jumlah cluster yang memberikan pemisahan terbaik antar kelompok data, sehingga model yang dibangun dapat lebih efektif. Berikut adalah **rekomendasi** tahapannya.
1. Gunakan Silhouette Score dan Elbow Method untuk menentukan jumlah cluster optimal.
2. Hitung Silhouette Score sebagai ukuran kualitas cluster.

In [ ]:
#Tahap Awal (Determinasi metode terbaik)
def hierarchical_clustering(data, method='ward', threshold=0.55):
    # Pemakaian subset demi 'efisiensi'
    if len(data) > 10000:  # Dataset untuk inspeksi demi 'efisiensi RAM'
        sample_indices = np.random.choice(len(data), 10000, replace=False)
        data_subset = data[sample_indices]
    else:
        data_subset = data

    Z = linkage(data_subset, method=method)
    clusters = fcluster(Z, t=threshold, criterion='distance')

    # Cek nilai kluster
    silhouette_avg = silhouette_score(data_subset, clusters)

    return clusters, silhouette_avg, Z

def adjust_clustering(data):
    methods = ['ward', 'single', 'complete', 'average']  # Metode memungkinkan
    best_silhouette = 0
    best_clusters = None
    best_method = None

    if len(data) > 10000:  # Perwakilan
        sample_indices = np.random.choice(len(data), 10000, replace=False)
        data_subset = data[sample_indices]
    else:
        data_subset = data

    for method in methods:
        print(f"Testing method: {method}")
        clusters, silhouette_avg, Z = hierarchical_clustering(data_subset, method=method)

        if silhouette_avg >= 0.55:
            print(f"Method: {method}, Silhouette score: {silhouette_avg:.2f}")
            if silhouette_avg > best_silhouette:
                best_silhouette = silhouette_avg
                best_clusters = clusters
                best_method = method

        # Hasil inspeksi
        dump((clusters, silhouette_avg, Z), f'{method}_checkpoint.joblib')

    return best_clusters, best_silhouette, best_method

def run_clustering_with_multiprocessing(data):
    num_cores = multiprocessing.cpu_count()

    # Use a subset of data to avoid memory issues
    if len(data) > 10000:
        sample_indices = np.random.choice(len(data), 10000, replace=False)
        data_subset = data[sample_indices]
    else:
        data_subset = data

    # Persiapan kluster
    results = Parallel(n_jobs=num_cores)(
        delayed(hierarchical_clustering)(data_subset, method) for method in ['ward', 'single', 'complete', 'average']
    )

    # Penyimpanan hasil untuk sementara
    dump(results, 'clustering_results.joblib')

    return results
    #ke persiapan data

In [ ]:
#ke persiapan data
nilrobust = rscaldfgbgnum
dfnilrobust = pd.DataFrame(nilrobust, columns=dfgabungnum.columns)
#Eksekusi nilai

In [ ]:
#cek hasil siluet
clusters, silhouette_avg, Z = hierarchical_clustering(dfnilrobust)

# Inspeksi nilai
if silhouette_avg >= 0.70:
    print(f"Achieved optimal silhouette score: {silhouette_avg:.2f}")
elif silhouette_avg >= 0.55:
    print("Niai minimal terpenuhi.")
else:
    print("Nilai membutuhkan penyetelan lanjut karena nilai belum oprimal.")
#ke eksekusi hasil

In [ ]:
#Eksekusi Nilai
best_clusters, best_silhouette, best_method = adjust_clustering(dfnilrobust)
print(f"Metode terbaik: {best_method}, Nilai siluet terbaik: {best_silhouette:.2f}")

Metode pengetesan : ward


## **c. Feature Selection (Opsional)**

Silakan lakukan feature selection jika Anda membutuhkan optimasi model clustering. Jika Anda menerapkan proses ini, silakan lakukan pemodelan dan evaluasi kembali menggunakan kolom-kolom hasil feature selection. Terakhir, bandingkan hasil performa model sebelum dan sesudah menerapkan feature selection.

In [ ]:
#Modifikasi dengan seleksi fitur


## **d. Visualisasi Hasil Clustering**

Setelah model clustering dilatih dan jumlah cluster optimal ditentukan, langkah selanjutnya adalah menampilkan hasil clustering melalui visualisasi.

Berikut adalah **rekomendasi** tahapannya.
1. Tampilkan hasil clustering dalam bentuk visualisasi, seperti grafik scatter plot atau 2D PCA projection.

In [ ]:
#Type your code here

## **e. Analisis dan Interpretasi Hasil Cluster**

Setelah melakukan clustering, langkah selanjutnya adalah menganalisis karakteristik dari masing-masing cluster berdasarkan fitur yang tersedia.

Berikut adalah **rekomendasi** tahapannya.
1. Analisis karakteristik tiap cluster berdasarkan fitur yang tersedia (misalnya, distribusi nilai dalam cluster).
2. Berikan interpretasi: Apakah hasil clustering sesuai dengan ekspektasi dan logika bisnis? Apakah ada pola tertentu yang bisa dimanfaatkan?

In [ ]:
#Type your code here

Tulis hasil interpretasinya di sini.
1. Cluster 1:
2. Cluster 2:
3. Cluster 3:

# **7. Mengeksport Data**

Simpan hasilnya ke dalam file CSV.